In [ ]:
# this notebook is used to gnereate the csv files to visualize results
# input: raw results generated from the code
# output: a csv file of UQ scores only, all scans are ordered
# modifield from jsd_score.ipynb
# three variables thats needs to be changed every time run this notebook:
# results_name, baselineScaIdx, and output_name
# please also comment or uncomment some code at block 8 for different tests

In [ ]:
results_name = '/home/beams/WZHENG/EventDetection/rareEventHEDM_v1/d64/res-0.20-30.csv'
baselineScanIdx = 375 # hard code here, change it for a new dataset
output_name = 'lshr_d64_488_embed_488_04_30.csv'

In [ ]:
import csv
import numpy as np
from scipy.spatial import distance

In [ ]:
# unrodered list of distribution, uq scores, and datasets (scans) list
dist         = []
uqResults    = []
datasetsList = []

In [ ]:
rowNumber = 0
with open(results_name, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        if rowNumber >= 1:
            distriTemp = []
            x = row[0].split(",")
            for i in range(len(x)-2):
                distriTemp.append(float(x[i]))
            uqResults.append(float(x[-2]))
            dist.append(distriTemp)
            datasetsList.append(x[-1])
        rowNumber += 1

In [ ]:
jsdResults  = []
jsdIdxDict  = {}
uqIdxDict   = {}
nameIdxDict = {}
presDict    = {}

baseDist = dist[0]

In [ ]:
for i in range(len(uqResults)):
    resultTmp = distance.jensenshannon(np.array(baseDist), np.array(dist[i]))
    jsdResults.append(resultTmp)
    if resultTmp == 0:
        tmpIdx = baselineScanIdx
    else: 
        tmpIdx = int(datasetsList[i].split(".")[0].split("_")[-1])
    jsdIdxDict[tmpIdx]  = resultTmp
    uqIdxDict[tmpIdx]   = uqResults[i]
    nameIdxDict[tmpIdx] = datasetsList[i]
    # need to obtain the pressure info:
    presDict[tmpIdx] = -10 # this is the adhoc for N.A. pressure
    for tmpStr in datasetsList[i].split("_"):
        if 'mpa' in tmpStr.lower():
            presDict[tmpIdx] = int(tmpStr[0:-3])
            break
        # if tmpStr.lower()[-1] == 'n':
        #     presDict[tmpIdx] = int(tmpStr[0:-1])
        #     break

In [ ]:
for key in presDict:
    if presDict[key] == -10:
        presDict[key] = 0

In [ ]:
presDict

In [ ]:
sortedJSD  = []
sortedIdx  = []
sortedName = []
sortedUQ   = []
sortedPres = []

for key in sorted(jsdIdxDict):
    sortedIdx.append(key)
    sortedJSD.append(jsdIdxDict[key])
    sortedUQ.append(uqIdxDict[key])
    sortedName.append(nameIdxDict[key])
    sortedPres.append(presDict[key])

In [ ]:
sortedIdx

In [ ]:
# now start to write data back to CSV files

In [ ]:
i = 0

with open(output_name, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ScanID', 'file name', 'loading size', 'UQ score', 'JSD score'])
    for key in sortedIdx:
        tmpRow = []
        tmpRow.append(sortedIdx[i])
        tmpRow.append(sortedName[i])
        tmpRow.append(sortedPres[i])
        tmpRow.append(sortedUQ[i])
        tmpRow.append(sortedJSD[i])
        i += 1
        writer.writerow(tmpRow)